In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
# VERSION NAME FOR SAVED MODEL FILES
VER = 1

# TRAIN RANDOM SEED
SEED = 42

# FILL NAN VALUE
NAN_VALUE = -127 # will fit in int8

# FOLDS PER MODEL
FOLDS = 5

In [2]:
train_df = pd.read_feather('train_data')
train_labels_df = pd.read_feather('train_labels')

In [3]:
test_df = pd.read_feather('test_data')

In [4]:
train_df.head(2)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217


In [5]:
len(train_df.customer_ID.unique())

458913

In [6]:
train_labels_df.head(2)

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0


In [7]:
len(train_labels_df)

458913

In [8]:
test_df.head(2)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631315,0.001912,0.010728,0.814497,0.007547,0.168651,0.009971,0.002347,...,NaN,NaN,NaN,NaN,0.004669,NaN,NaN,NaN,0.008281,NaN
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.587042,0.005275,0.011026,0.810848,0.001817,0.241389,0.000166,0.009132,...,NaN,NaN,NaN,0.000142,0.004940,0.009021,NaN,0.003695,0.003753,0.00146


In [10]:
len(test_df)

11363762

In [13]:
len(test_df.customer_ID.unique())

924621

In [14]:
len(test_df.customer_ID)

11363762

In [21]:
len(train_labels_df.customer_ID.unique()), len(train_labels_df.customer_ID)

(458913, 458913)

In [22]:
train_test_df = pd.concat([train_df, test_df])  # memory 이슈로 합쳐서 진행하기엔 힘들어보임 -> 같은 전처리 과정을 함수화하여 처리하는 것이 나아보임

: 

: 

# Process and Feature Engineer Train Data

In [18]:
train_df.dtypes[train_df.dtypes == object]

customer_ID    object
S_2            object
D_63           object
D_64           object
dtype: object

In [5]:
all_cols = [c for c in list(train_df.columns) if c not in ['customer_ID','S_2']]
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
num_features = [col for col in all_cols if col not in cat_features]

test_num_agg = train_df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

test_cat_agg = train_df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]


shape after engineering (458913, 918)


# Train XGB